TN Gov publishes the list of covid beds in each district in this page. This effort is to improve search easier

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import json

In [2]:
covid_beds_url = "https://stopcorona.tn.gov.in/beds.php"

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
import urllib.request
page = urllib.request.urlopen(covid_beds_url)

In [5]:
soup = BeautifulSoup(page.read())

In [6]:
table = soup.find("table", {"id": "dtBasicExample"})
df = pd.read_html(str(table))[0]

In [7]:
df.head(1)

S.NO  District   Institution COVID BEDS                  \
  S.NO  District   Institution      Total Occupied Vacant   
0    1  Ariyalur  A.S Hospital         30       29      1   

  OXYGEN SUPPORTED BEDS                 NON-OXYGEN SUPPORTED BEDS  ...         \
                  Total Occupied Vacant                     Total  ... Vacant   
0                     8        8      0                        12  ...      0   

  ICU BEDS                 VENTILATOR                         Last updated  \
     Total Occupied Vacant      Total Occupied Vacant         Last updated   
0       10        9      1          2        2      0  2021-05-12 09:36:55   

  Contact Number     Remarks  
  Contact Number     Remarks  
0            0.0  12.05.2021  

[1 rows x 21 columns]

In [8]:
correct_cols = ["S.NO", "District", "Institution", "Last updated", "Contact Number","Remarks"]

new_cols = [f"{column[0]}_{column[1]}" if column[0]!=column[1] else column[0] for column in df.columns]

In [9]:
df.columns = new_cols

In [10]:
df['Last updated'] = pd.to_datetime(df['Last updated'])
df['Contact Number'] = df['Contact Number'].astype(str)
df['Remarks'] = df['Remarks'].astype(str)

In [11]:
df.head(2)

,S.NO,District,Institution,COVID BEDS_Total,COVID BEDS_Occupied,COVID BEDS_Vacant,OXYGEN SUPPORTED BEDS_Total,OXYGEN SUPPORTED BEDS_Occupied,OXYGEN SUPPORTED BEDS_Vacant,NON-OXYGEN SUPPORTED BEDS_Total,...,NON-OXYGEN SUPPORTED BEDS_Vacant,ICU BEDS_Total,ICU BEDS_Occupied,ICU BEDS_Vacant,VENTILATOR_Total,VENTILATOR_Occupied,VENTILATOR_Vacant,Last updated,Contact Number,Remarks
0,1,Ariyalur,A.S Hospital,30,29,1,8,8,0,12,...,0,10,9,1,2,2,0,2021-05-12 09:36:55,0.0,12.05.2021
1,2,Ariyalur,Golden Hospital,31,30,1,23,23,0,3,...,1,5,5,0,5,5,0,2021-05-12 10:10:19,9487576493.0,Report 12/05/2021.


In [12]:
geo_data = []
for district in df['District'].unique():
    geo_dict = {
        "latitude" : 0,
        "longitude" : 0,
        "district" : district
    }
    geo_data.append(geo_dict)

In [13]:
'''import json
with open('geo.json', 'w') as out:
    json.dump({"geo_data": geo_data}, indent=4, fp=out)'''

'import json\nwith open(\'geo.json\', \'w\') as out:\n    json.dump({"geo_data": geo_data}, indent=4, fp=out)'

In [14]:
with open('geo_districts.json',"r") as f:
    data = json.loads(f.read())

In [15]:
df_geo = pd.DataFrame(data.get('geo_data'))
print(df_geo.shape)
df_geo.head()

(35, 3)


,latitude,longitude,district
0,11.140059,79.078629,Ariyalur
1,12.693620,79.980499,Chengalpattu
2,13.082680,80.270721,Chennai
3,12.834174,79.703644,Kancheepuram
4,11.016844,76.955833,Coimbatore


In [16]:
df_district = df.groupby('District', as_index=False).sum()
print(df_district.shape)

(35, 17)


In [17]:
df_district.head()

,District,S.NO,COVID BEDS_Total,COVID BEDS_Occupied,COVID BEDS_Vacant,OXYGEN SUPPORTED BEDS_Total,OXYGEN SUPPORTED BEDS_Occupied,OXYGEN SUPPORTED BEDS_Vacant,NON-OXYGEN SUPPORTED BEDS_Total,NON-OXYGEN SUPPORTED BEDS_Occupied,NON-OXYGEN SUPPORTED BEDS_Vacant,ICU BEDS_Total,ICU BEDS_Occupied,ICU BEDS_Vacant,VENTILATOR_Total,VENTILATOR_Occupied,VENTILATOR_Vacant
0,Ariyalur,969,101,90,11,53,47,6,24,21,3,24,22,2,9,7,2
1,Chengalpattu,4190,679,647,32,448,446,2,148,119,29,83,82,1,44,40,4
2,Chengalpet,320,25,25,0,17,17,0,7,7,0,1,1,0,1,1,0
3,Chennai,45093,9767,8773,994,5334,5281,53,3057,2120,937,1376,1372,4,616,602,14
4,Coimbatore,7947,3595,3377,218,1861,1824,37,1417,1236,181,324,317,7,177,177,0


In [18]:
df_district = pd.merge(
    df_district,
    df_geo,
    how="inner",
    left_on="District",
    right_on="district"
)

In [19]:
df_district.head()

,District,S.NO,COVID BEDS_Total,COVID BEDS_Occupied,COVID BEDS_Vacant,OXYGEN SUPPORTED BEDS_Total,OXYGEN SUPPORTED BEDS_Occupied,OXYGEN SUPPORTED BEDS_Vacant,NON-OXYGEN SUPPORTED BEDS_Total,NON-OXYGEN SUPPORTED BEDS_Occupied,NON-OXYGEN SUPPORTED BEDS_Vacant,ICU BEDS_Total,ICU BEDS_Occupied,ICU BEDS_Vacant,VENTILATOR_Total,VENTILATOR_Occupied,VENTILATOR_Vacant,latitude,longitude,district
0,Ariyalur,969,101,90,11,53,47,6,24,21,3,24,22,2,9,7,2,11.140059,79.078629,Ariyalur
1,Chengalpattu,4190,679,647,32,448,446,2,148,119,29,83,82,1,44,40,4,12.693620,79.980499,Chengalpattu
2,Chengalpet,320,25,25,0,17,17,0,7,7,0,1,1,0,1,1,0,12.684089,79.983637,Chengalpet
3,Chennai,45093,9767,8773,994,5334,5281,53,3057,2120,937,1376,1372,4,616,602,14,13.082680,80.270721,Chennai
4,Coimbatore,7947,3595,3377,218,1861,1824,37,1417,1236,181,324,317,7,177,177,0,11.016844,76.955833,Coimbatore


In [20]:
# Save all files
df.to_csv("tn_covid_beds.csv")
df_district.to_csv("tn_covid_district_beds.csv")